In [4]:
# Se necesita instalar Augmentor. En este caso lo instalo en Colab
!pip install Augmentor

In [36]:
# Importo las bibliotecas que voy a necesitar para el proceso
from bs4 import BeautifulSoup
import requests
import re
import Augmentor
from google.colab import drive

In [38]:
# Definir los directorios de origen y destino de los datos
# En mi caso, monto los recursos de Drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Compruebo que todo haya ido bien revisando el contenido del punto de montaje
!ls /content/drive/MyDrive/Maiz

In [42]:
# Defino las variables de entorno. En este caso, el directorio de origen de las imágenes
RUTA_DESCARGA_JPGS = "/content/drive/MyDrive/Maiz/"

In [44]:
!ls RUTA_DESCARGA_JPGS

ls: cannot access 'RUTA_DESCARGA_JPGS': No such file or directory


### Este código muestra todos los fichero JPG del respositorio. Sin embargo, esas URLs tal como son mostradas NO son las de descarga. Para descargar el fichero como tal, se ha de buscar el fichero binario (RAW)

In [45]:
https = 'https://github.com' # Encabezado de la url de descara. No dada en el HTML
# Directorio donde están las fotos
url_base = 'https://github.com/aantolinez/JORGE_WASHINGTON/blob/a7f02346df9dd7e8f02726f742ad15970784b03f/Imagenes/NoIR/'
ext = 'jpg' # Extensión de los ficheros buscados

params={'raw': 'true'} # Parámetros para descargar el binario

In [46]:
page = requests.get(url_base).text
soup = BeautifulSoup(page, 'html.parser')

In [49]:
RUTA_DESCARGA_JPGS = "/content/drive/MyDrive/Maiz/"
for link in soup.find_all('a'):
  txt = link.get('href')
  x = re.search("jpg$", txt)
  if x:
    partes = txt.rsplit("/") # Separo la URL devuelta mediante "/" y lo guardo en una lista, cuyo último elemento es el nombre del fichero
    nombre_fichero = partes[(len(partes) - 1)] # Accedo al último elemento, que es el nombre del fichero
    fichero_url = https + txt # Concateno el encabezado y el cuerpo de la url

    r = requests.get(fichero_url, stream = True)
    
    with open(RUTA_DESCARGA_JPGS + nombre_fichero, "wb") as file:
      for block in r.iter_content(chunk_size = 1024):
        if block:
          file.write(block)
          #print(fichero_url, nombre_fichero)

### Esto puede ser lo que realmente te interese a ti.
* Define la ruta de tus ficheros. Si los tienes en tu disco local, sería algo como C:\\ruta\\a\\las\\imagenes\\

In [ ]:
#RUTA_DESCARGA_JPGS = "/content/drive/MyDrive/Maiz/"
p = Augmentor.Pipeline(RUTA_DESCARGA_JPGS) # Inicializamos el directorio de origen y de salida de los datos. Pipeline

In [ ]:
# Establecemos las acciones que queremos realizar sobre la base de datos
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)

# Primero, volteo horizontal:
p.flip_left_right(probability=0.4)

# Segundo, volteo vertical:
p.flip_top_bottom(probability=0.8)

# Tercero, añadimos una rotación de 90 grados:
p.rotate90(probability=0.1)

In [ ]:
# Establecemos el tamaño de la muestra, (número de imágenes), que queremos generar
# En este caso, genero 100 nuevas imágenes ... Tu puedes generar la que necesites
num_of_samples = int(1e2) # 1e2 = 100, 1e3 = 1000

p.sample(num_of_samples) # Los ficheros se depositarán en RUTA_DESCARGA_JPGS/output